# Chatbot Memory

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant. Answer all questions to the best 
        of your ability."""),
    ("placeholder", "{messages}"),
])

model = ChatOllama(model="qwen:0.5b", temperature=0)

chain = prompt | model

chain.invoke({
    "messages": [
        ("human","""Translate this sentence from English to French: I love 
            programming."""),
        ("ai", "J'adore programmer."),
        ("human", "What did you just say?"),
    ],
})

AIMessage(content='I just said "j\'adore programmer."', additional_kwargs={}, response_metadata={'model': 'qwen:0.5b', 'created_at': '2025-08-07T13:19:59.2853048Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2454663500, 'load_duration': 1659153100, 'prompt_eval_count': 64, 'prompt_eval_duration': 471026800, 'eval_count': 11, 'eval_duration': 319434200, 'model_name': 'qwen:0.5b'}, id='run--c03d484c-e716-449c-8e44-05d2aa6d24d3-0', usage_metadata={'input_tokens': 64, 'output_tokens': 11, 'total_tokens': 75})

## Langraph (for memory)

* State: Data
* Nodes: Process States
* Edges: Connect Nodes

In [6]:
from typing import Annotated, TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` 
    # function in the annotation defines how this state should 
    # be updated (in this case, it appends new messages to the 
    # list, rather than replacing the previous messages)
	messages: Annotated[list, add_messages]

builder = StateGraph(State)

In [7]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="qwen:0.5b")

def chatbot(state: State):
    answer = model.invoke(state["messages"])
    return {"messages": [answer]}

# The first argument is the unique node name
# The second argument is the function or Runnable to run
builder.add_node("chatbot", chatbot)

In [8]:
builder.add_edge(START, 'chatbot')
builder.add_edge('chatbot', END)

graph = builder.compile()

In [9]:
graph.get_graph().draw_mermaid()

'---\nconfig:\n  flowchart:\n    curve: linear\n---\ngraph TD;\n\t__start__([<p>__start__</p>]):::first\n\tchatbot(chatbot)\n\t__end__([<p>__end__</p>]):::last\n\t__start__ --> chatbot;\n\tchatbot --> __end__;\n\tclassDef default fill:#f2f0ff,line-height:1.2\n\tclassDef first fill-opacity:0\n\tclassDef last fill:#bfb6fc\n'

In [10]:
from langchain_core.messages import HumanMessage

In [11]:
input = {"messages": [HumanMessage('hi!')]}
for chunk in graph.stream(input):
    print(chunk)

{'chatbot': {'messages': [AIMessage(content='Hello there, how may I assist you today?', additional_kwargs={}, response_metadata={'model': 'qwen:0.5b', 'created_at': '2025-08-07T13:20:00.1620944Z', 'done': True, 'done_reason': 'stop', 'total_duration': 472404200, 'load_duration': 34020500, 'prompt_eval_count': 10, 'prompt_eval_duration': 98713100, 'eval_count': 11, 'eval_duration': 337698500, 'model_name': 'qwen:0.5b'}, id='run--00ee30b3-1bdb-43ee-a28d-311242c12587-0', usage_metadata={'input_tokens': 10, 'output_tokens': 11, 'total_tokens': 21})]}}


### Add memory

In [12]:
from langgraph.checkpoint.memory import MemorySaver

graph = builder.compile(checkpointer=MemorySaver())

In [13]:
thread1 = {"configurable": {"thread_id": "1"}} #Identify the conversation
result_1 = graph.invoke(
    { "messages": [HumanMessage("hi, my name is Jack!")] }, 
    thread1
)

In [14]:
result_1

{'messages': [HumanMessage(content='hi, my name is Jack!', additional_kwargs={}, response_metadata={}, id='0339de65-5bf7-4129-a62b-a35e5411fcaa'),
  AIMessage(content='Hello Jack! How may I assist you today?', additional_kwargs={}, response_metadata={'model': 'qwen:0.5b', 'created_at': '2025-08-07T13:20:00.5505043Z', 'done': True, 'done_reason': 'stop', 'total_duration': 348238700, 'load_duration': 24344300, 'prompt_eval_count': 15, 'prompt_eval_duration': 96849000, 'eval_count': 11, 'eval_duration': 225521900, 'model_name': 'qwen:0.5b'}, id='run--ee281e32-a59b-4457-8b99-e4214be7db0d-0', usage_metadata={'input_tokens': 15, 'output_tokens': 11, 'total_tokens': 26})]}

In [15]:
result_2 = graph.invoke(
    { "messages": [HumanMessage("what is my name?")] }, 
    thread1
)

In [16]:
result_2

{'messages': [HumanMessage(content='hi, my name is Jack!', additional_kwargs={}, response_metadata={}, id='0339de65-5bf7-4129-a62b-a35e5411fcaa'),
  AIMessage(content='Hello Jack! How may I assist you today?', additional_kwargs={}, response_metadata={'model': 'qwen:0.5b', 'created_at': '2025-08-07T13:20:00.5505043Z', 'done': True, 'done_reason': 'stop', 'total_duration': 348238700, 'load_duration': 24344300, 'prompt_eval_count': 15, 'prompt_eval_duration': 96849000, 'eval_count': 11, 'eval_duration': 225521900, 'model_name': 'qwen:0.5b'}, id='run--ee281e32-a59b-4457-8b99-e4214be7db0d-0', usage_metadata={'input_tokens': 15, 'output_tokens': 11, 'total_tokens': 26}),
  HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}, id='8450aba2-e530-4b16-9ab1-11ec3ac8b797'),
  AIMessage(content='Your name is Jack.', additional_kwargs={}, response_metadata={'model': 'qwen:0.5b', 'created_at': '2025-08-07T13:20:00.8221477Z', 'done': True, 'done_reason': 'stop', 'total_

In [17]:
graph.get_state(thread1)

StateSnapshot(values={'messages': [HumanMessage(content='hi, my name is Jack!', additional_kwargs={}, response_metadata={}, id='0339de65-5bf7-4129-a62b-a35e5411fcaa'), AIMessage(content='Hello Jack! How may I assist you today?', additional_kwargs={}, response_metadata={'model': 'qwen:0.5b', 'created_at': '2025-08-07T13:20:00.5505043Z', 'done': True, 'done_reason': 'stop', 'total_duration': 348238700, 'load_duration': 24344300, 'prompt_eval_count': 15, 'prompt_eval_duration': 96849000, 'eval_count': 11, 'eval_duration': 225521900, 'model_name': 'qwen:0.5b'}, id='run--ee281e32-a59b-4457-8b99-e4214be7db0d-0', usage_metadata={'input_tokens': 15, 'output_tokens': 11, 'total_tokens': 26}), HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}, id='8450aba2-e530-4b16-9ab1-11ec3ac8b797'), AIMessage(content='Your name is Jack.', additional_kwargs={}, response_metadata={'model': 'qwen:0.5b', 'created_at': '2025-08-07T13:20:00.8221477Z', 'done': True, 'done_reason': 

In [18]:
graph.update_state(thread1, { "messages": [HumanMessage("I Like LLMs!")] })

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f073913-8cd0-6de4-8005-f2aeca29300f'}}

In [19]:
graph.get_state(thread1)

StateSnapshot(values={'messages': [HumanMessage(content='hi, my name is Jack!', additional_kwargs={}, response_metadata={}, id='0339de65-5bf7-4129-a62b-a35e5411fcaa'), AIMessage(content='Hello Jack! How may I assist you today?', additional_kwargs={}, response_metadata={'model': 'qwen:0.5b', 'created_at': '2025-08-07T13:20:00.5505043Z', 'done': True, 'done_reason': 'stop', 'total_duration': 348238700, 'load_duration': 24344300, 'prompt_eval_count': 15, 'prompt_eval_duration': 96849000, 'eval_count': 11, 'eval_duration': 225521900, 'model_name': 'qwen:0.5b'}, id='run--ee281e32-a59b-4457-8b99-e4214be7db0d-0', usage_metadata={'input_tokens': 15, 'output_tokens': 11, 'total_tokens': 26}), HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}, id='8450aba2-e530-4b16-9ab1-11ec3ac8b797'), AIMessage(content='Your name is Jack.', additional_kwargs={}, response_metadata={'model': 'qwen:0.5b', 'created_at': '2025-08-07T13:20:00.8221477Z', 'done': True, 'done_reason': 

### Modify Chat History

* trimming (window)
* filtering
* merging

##### Trimming

In [20]:
from langchain_core.messages import SystemMessage, trim_messages, AIMessage, HumanMessage
from langchain_ollama import ChatOllama
from langchain_core.messages.utils import count_tokens_approximately

trimmer = trim_messages(
    max_tokens=60,
    strategy="last",
    token_counter=count_tokens_approximately,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="what's 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="what's 2 + 2", additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [21]:
trimmer = trim_messages(
    max_tokens=5,
    strategy="last",
    token_counter=len, #Each message is a token
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="what's 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

##### Filtering (Filter by type, id or name)

##### By Type

In [22]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    filter_messages,
)

messages = [
    SystemMessage("you are a good assistant", id="1"),
    HumanMessage("example input", id="2", name="example_user"),
    AIMessage("example output", id="3", name="example_assistant"),
    HumanMessage("real input", id="4", name="bob"),
    AIMessage("real output", id="5", name="alice"),
]

filter_messages(messages, include_types="human")

[HumanMessage(content='example input', additional_kwargs={}, response_metadata={}, name='example_user', id='2'),
 HumanMessage(content='real input', additional_kwargs={}, response_metadata={}, name='bob', id='4')]

##### By name (with exclude)

In [23]:
filter_messages(messages, exclude_names=["example_user", "example_assistant"])

[SystemMessage(content='you are a good assistant', additional_kwargs={}, response_metadata={}, id='1'),
 HumanMessage(content='real input', additional_kwargs={}, response_metadata={}, name='bob', id='4'),
 AIMessage(content='real output', additional_kwargs={}, response_metadata={}, name='alice', id='5')]

##### By type and id (Mix include, exclude)

In [24]:
filter_messages(
    messages, 
    include_types=[HumanMessage, AIMessage], 
    exclude_ids=["3"]
)

[HumanMessage(content='example input', additional_kwargs={}, response_metadata={}, name='example_user', id='2'),
 HumanMessage(content='real input', additional_kwargs={}, response_metadata={}, name='bob', id='4'),
 AIMessage(content='real output', additional_kwargs={}, response_metadata={}, name='alice', id='5')]

In [25]:
filter_ = filter_messages(exclude_names=["example_user", "example_assistant"])

chain = filter_ | model

In [32]:
messages = [
    SystemMessage("you are a good assistant", id="1"),
    HumanMessage("example input", id="2", name="example_user"),
    AIMessage("example output", id="3", name="example_assistant"),
    HumanMessage("real input", id="4", name="bob"),
    AIMessage("real output", id="5", name="alice"),
    HumanMessage("Tell me something", id="6", name="random"),
]
chain.invoke(messages)

AIMessage(content='There is no specific information that can be told. However, if you could provide more context or specify what you would like to know, I would be happy to assist you.', additional_kwargs={}, response_metadata={'model': 'qwen:0.5b', 'created_at': '2025-08-07T13:24:22.437432Z', 'done': True, 'done_reason': 'stop', 'total_duration': 968686400, 'load_duration': 26660600, 'prompt_eval_count': 32, 'prompt_eval_duration': 91539100, 'eval_count': 36, 'eval_duration': 845563300, 'model_name': 'qwen:0.5b'}, id='run--3e43d971-45f8-4d38-a417-6ab7d08e165d-0', usage_metadata={'input_tokens': 32, 'output_tokens': 36, 'total_tokens': 68})

##### Merging (Same type consecutive)

In [42]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    merge_message_runs,
)

messages = [
    SystemMessage("you're a good assistant."),
    SystemMessage("you always respond with a joke."),
    HumanMessage(
        [{"type": "text", "text": "i wonder why it's called langchain"}]
    ),
    HumanMessage([{ "type": "text", "text":"and who is harrison chasing anyway"}]),
    AIMessage(
        [{ "type": "text", "text":'''Well, I guess they thought "WordRope" and "SentenceString" just 
        didn\'t have the same ring to it!'''}]
    ),
    AIMessage([{ "type": "text", "text":"""Why, he's probably chasing after the last cup of coffee in the 
        office!"""}]),
    HumanMessage([{ "type": "text", "text":"Tell me a joke"}]),
]

merge_message_runs(messages)

[SystemMessage(content="you're a good assistant.\nyou always respond with a joke.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content=[{'type': 'text', 'text': "i wonder why it's called langchain"}, {'type': 'text', 'text': 'and who is harrison chasing anyway'}], additional_kwargs={}, response_metadata={}),
 AIMessage(content=[{'type': 'text', 'text': 'Well, I guess they thought "WordRope" and "SentenceString" just \n        didn\'t have the same ring to it!'}, {'type': 'text', 'text': "Why, he's probably chasing after the last cup of coffee in the \n        office!"}], additional_kwargs={}, response_metadata={}),
 HumanMessage(content=[{'type': 'text', 'text': 'Tell me a joke'}], additional_kwargs={}, response_metadata={})]

In [43]:
merger = merge_message_runs()
chain = merger | model

In [44]:
chain.invoke(messages)

AIMessage(content="Here's a classic one for you:\n\nWhy was the math book sad?\n\nBecause it had too many problems.", additional_kwargs={}, response_metadata={'model': 'qwen:0.5b', 'created_at': '2025-08-07T13:34:09.9548131Z', 'done': True, 'done_reason': 'stop', 'total_duration': 660641500, 'load_duration': 23518300, 'prompt_eval_count': 110, 'prompt_eval_duration': 77368000, 'eval_count': 23, 'eval_duration': 556064300, 'model_name': 'qwen:0.5b'}, id='run--07e45ec1-4454-4cfd-83aa-3bf11a11f41a-0', usage_metadata={'input_tokens': 110, 'output_tokens': 23, 'total_tokens': 133})